# Temporal Segmentation Test Script

## Setup and Data Pre-Processing
- We need the data to be a series of images at set time steps so if we are passing in a video file or something like that, we need to convert it to a set of discrete images for SAM

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import sys
import time
from IPython.display import display, clear_output
import json

def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
def show_anns_reg(anns):
    if len(anns) == 0:
        return
    # sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((anns[0]['segmentation'].shape[0], anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
# make a list of colors
def random_colors(N):
    np.random.seed(1)
    intensity = 0.6
    color = np.concatenate([np.random.random(3), np.array([intensity])], axis=0)
    colors = np.array([color])
    for i in range(N-1):
        color = np.concatenate([np.random.random(3), np.array([intensity])], axis=0)
        colors = np.concatenate([colors, [color]], axis=0)
    return colors

color_list = random_colors(100)

def show_mask(mask, ax, random_color=False, color=None):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    elif color is not None:
        color = color
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))    

In [ ]:
# getting the hu moments of the mask for shape dissimilarity
def hu_moments(mask):
    mask = mask.astype(np.uint8)
    moments = cv2.moments(mask)
    hu_moments = cv2.HuMoments(moments)
    return hu_moments

# calculating the shape dissimilarity
def shape_dissimilarity(hu_moments1, hu_moments2):
    diff = np.abs(hu_moments1 - hu_moments2)
    dissimilarity = np.mean(diff)
    return dissimilarity

In [ ]:
def hungarian_algorithm(cost_matrix):
    num_rows = len(cost_matrix)
    num_cols = len(cost_matrix[0])

    # Step 1: Subtract the minimum value from each row
    for i in range(num_rows):
        min_value = min(cost_matrix[i])
        for j in range(num_cols):
            cost_matrix[i][j] -= min_value

    # Step 2: Subtract the minimum value from each column
    for j in range(num_cols):
        min_value = min(cost_matrix[i][j] for i in range(num_rows))
        for i in range(num_rows):
            cost_matrix[i][j] -= min_value

    # Step 3: Find a complete matching by selecting zero entries
    matching = []
    for i in range(num_rows):
        for j in range(num_cols):
            if cost_matrix[i][j] == 0 and not any(j in m for m in matching):
                matching.append((i, j))
                break

    # Step 4: If the matching is not complete, update the matrix and repeat steps 3-4
    while len(matching) < min(num_rows, num_cols):
        # Find the minimum uncovered value
        min_uncovered = float('inf')
        for i in range(num_rows):
            if not any(i == m[0] for m in matching):
                for j in range(num_cols):
                    if not any(j == m[1] for m in matching):
                        min_uncovered = min(min_uncovered, cost_matrix[i][j])

        # Subtract the minimum uncovered value from each uncovered row
        for i in range(num_rows):
            if not any(i == m[0] for m in matching):
                for j in range(num_cols):
                    cost_matrix[i][j] -= min_uncovered

        # Add the minimum uncovered value to each covered column
        for j in range(num_cols):
            if any(j == m[1] for m in matching):
                for i in range(num_rows):
                    cost_matrix[i][j] += min_uncovered

        # Find a complete matching by selecting zero entries
        for i in range(num_rows):
            for j in range(num_cols):
                if cost_matrix[i][j] == 0 and not any(j in m for m in matching):
                    matching.append((i, j))
                    break

    return matching

In [ ]:
# get matching masks to previous frame and keep matching segmentations based on IOU using Hungarian algorithm
# good weights were 0.4, 0.05, 0.2, 0.2
# def get_matching_masks(prev_masks, curr_masks, iou_weight=0.3, dist_weight=0.1, size_diff_weight=0.3, shape_dissimilarity_weight=0.3):
def get_matching_masks(prev_masks, curr_masks, iou_weight=0.45, dist_weight=0.1, size_diff_weight=0.2, shape_dissimilarity_weight=0.4):
    n_prev_masks = len(prev_masks)
    n_curr_masks = len(curr_masks)
    
    # if there are no masks in the previous frame, then return all the masks in the current frame
    if n_prev_masks == 0:
        return list(range(n_curr_masks))
    
    # if there are no masks in the current frame, then return an empty list
    if n_curr_masks == 0:
        return []
    
    # compute the distance between each mask centroid in prev_masks and each mask centroid in curr_mask as a percentage of width and height
    distance_matrix = np.zeros((n_prev_masks, n_curr_masks))
    # compute the percent size difference between each mask in prev_masks and the corresponding mask in curr_masks
    percent_size_diff = np.zeros((n_prev_masks, n_curr_masks))
    # calculate IOU between all pairs of masks
    shape_dissimilarity_matrix = np.zeros((n_prev_masks, n_curr_masks))
    iou_matrix = np.zeros((n_prev_masks, n_curr_masks))
    cost_matrix = np.zeros((n_prev_masks, n_curr_masks))
    for i in range(n_prev_masks):
        for j in range(n_curr_masks):
            hu1 = hu_moments(prev_masks[i])
            hu2 = hu_moments(curr_masks[j])
            iou_matrix[i][j] = iou(prev_masks[i], curr_masks[j])
            shape_dissimilarity_matrix[i][j] = shape_dissimilarity(hu1, hu2)
            prev_centroid = np.mean(np.argwhere(prev_masks[i]), axis=0)
            curr_centroid = np.mean(np.argwhere(curr_masks[j]), axis=0)
            distance_matrix[i][j] = np.linalg.norm(prev_centroid - curr_centroid) / np.sqrt(prev_masks[i].shape[0] * prev_masks[i].shape[1])
            prev_area = np.sum(prev_masks[i])
            curr_area = np.sum(curr_masks[j])
            percent_size_diff[i][j] = (abs(prev_area - curr_area) / prev_area)
            # cost_matrix[i][j] = -iou_matrix[i][j] * iou_weight + distance_matrix[i][j] * dist_weight + percent_size_diff[i][j] * size_diff_weight + shape_dissimilarity_matrix[i][j] * shape_dissimilarity_weight
            cost_matrix[i][j] = (1-iou_matrix[i][j]) * iou_weight + distance_matrix[i][j] * dist_weight + percent_size_diff[i][j] * size_diff_weight + shape_dissimilarity_matrix[i][j] * shape_dissimilarity_weight
    
    # use scipy to solve the assignment problem
    from scipy.optimize import linear_sum_assignment
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    # get the matching pairs of masks
    matching_pairs = []
    for i in range(len(row_ind)):
        # if iou_matrix[row_ind[i]][col_ind[i]] > iou_thresh:
        matching_pairs.append((row_ind[i], col_ind[i]))
    # matching_pairs = hungarian_algorithm(cost_matrix)

    # print weighted shape dissimilarity, iou, distance and size difference for each matching pair
    for i, j in matching_pairs:
        print(f'Weighted Shape Dissimilarity: {shape_dissimilarity_matrix[i][j] * shape_dissimilarity_weight}, IOU: {iou_matrix[i][j] * iou_weight}, Distance: {distance_matrix[i][j] * dist_weight}, Size Difference: {percent_size_diff[i][j] * size_diff_weight}')
    
    return matching_pairs

# calculate the intersection over union (IOU) of two masks
def iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    return np.sum(intersection) / np.sum(union)

In [ ]:
# function to get the bounding box given a mask, not using openCV
def get_box(segmentation_mask):
    rows = len(segmentation_mask)
    if rows == 0:
        return None
    cols = len(segmentation_mask[0])
    
    # Initialize bounding box coordinates
    min_row, min_col = rows, cols
    max_row, max_col = 0, 0
    
    # Iterate through the segmentation mask to find True values
    for row in range(rows):
        for col in range(cols):
            if segmentation_mask[row][col]:
                min_row = min(min_row, row)
                min_col = min(min_col, col)
                max_row = max(max_row, row)
                max_col = max(max_col, col)
    
    # If no True values found, return None
    if min_row == rows or min_col == cols:
        return None
    
    return [min_col, min_row, max_col, max_row]

In [ ]:
def find_centroid(mask):
    mask = mask.astype(np.uint8)
    M = cv2.moments(mask)
    if M["m00"] == 0:
        return np.array([0, 0])
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    return np.array([cX, cY])

def find_centroid_2(mask):
    # using np.argwhere and np.mean
    return np.mean(np.argwhere(mask), axis=0)

In [ ]:
# Given an array of mask numbers, combine all the masks and return the combined mask
def combine_masks(mask_nums, masks):
    combined_mask = np.zeros(masks[0]['segmentation'].shape, dtype=bool)
    for i in mask_nums:
        combined_mask = np.logical_or(combined_mask, masks[i]['segmentation'])
    return combined_mask

# Given an np.array of shape (mask_num, width, height), return the combined mask
def combine_all_masks(masks):
    combined_mask = np.zeros(masks[0].shape, dtype=bool)
    for mask in masks:
        combined_mask = np.logical_or(combined_mask, mask)
    return combined_mask

In [ ]:
# function to get annotations from the json file
def get_annotations(json_file):
    # Read the JSON data from the file
    data = None
    with open(json_file, 'r') as file:
        data = json.load(file)

    annotations = []
    # Iterate over each image in the JSON data
    for image_data in data:
        image_path = image_data['image']
        index = image_path.index("test-images")
        image_path = image_path[index:]
        image_path = image_path.replace("X", "/")
        original_width = image_data['label'][0]['original_width']
        original_height = image_data['label'][0]['original_height']
        
        # Create a dictionary to store the bounding boxes for each label
        bounding_boxes = {}
        
        # Iterate over each annotation for the current image
        for annotation in image_data['label']:
            label = annotation['rectanglelabels'][0]
            
            # Extract the bounding box coordinates
            x = annotation['x']
            y = annotation['y']
            width = annotation['width']
            height = annotation['height']
            
            # Scale the coordinates to the original image dimensions
            x_scaled = int(x * original_width / 100)
            y_scaled = int(y * original_height / 100)
            width_scaled = int(width * original_width / 100)
            height_scaled = int(height * original_height / 100)
            
            # Calculate the xyxy coordinates
            x1 = x_scaled
            y1 = y_scaled
            x2 = x_scaled + width_scaled
            y2 = y_scaled + height_scaled
            
            # Add the bounding box coordinates to the dictionary
            if label not in bounding_boxes:
                bounding_boxes[label] = []
            bounding_boxes[label].append([x1, y1, x2, y2])
        
        annotations.append((image_path, bounding_boxes))

    ann_np = np.array(annotations)
    return ann_np

In [ ]:
# function that takes in a mask and returns the mask with the largest contiguous section still kept in the mask and nothing else
def get_largest_contiguous(mask):
    # get the largest contiguous section of the mask
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask.astype(np.uint8), connectivity=8)
    largest_label = 1
    max_area = stats[1, cv2.CC_STAT_AREA]
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] > max_area:
            max_area = stats[i, cv2.CC_STAT_AREA]
            largest_label = i
    mask = np.array(labels == largest_label, dtype=bool)
    return mask

In [ ]:
image_dir = '/home/yashas/Documents/thesis/segment-anything/test-images/group_0/traj0/images0/'
image_prefix = 'im_'
image_suffix = '.jpg'

In [ ]:
big_plot_dim = 8

In [ ]:
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

large_sam_checkpoint = "/home/yashas/Documents/thesis/segment-anything/sam_vit_h_4b8939.pth"
large_model_type = "vit_h"

# small_sam_checkpoint = "/home/yashas/Documents/thesis/segment-anything/sam_vit_b_01ec64.pth"
# small_model_type = "vit_b"

device = "cuda"

sam = sam_model_registry[large_model_type](checkpoint=large_sam_checkpoint)
# sam = sam_model_registry[small_model_type](checkpoint=small_sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

## First image visualization and segmentation

In [ ]:
image = cv2.imread(image_dir + image_prefix + '0' + image_suffix)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
predictor.set_image(image)

In [ ]:
plt.figure(figsize=(big_plot_dim, big_plot_dim))
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
# read testset_annotations.json file and get the annotations
json_file = "/home/yashas/Documents/thesis/thesis_working/testset_annotations.json"
annotations = get_annotations(json_file)

image_path = image_dir + image_prefix + '0' + image_suffix
image_path = image_path[image_path.index("test-images"):]

# get the annotations for the current image
curr_ann = None
for ann in annotations:
    if ann[0] == image_path:
        curr_ann = ann
        break
print(curr_ann)

# getting the bounding boxes and centroid points for each label
boxes = []
boxes.append(curr_ann[1]['gripper'][0])
boxes.append(curr_ann[1]['table'][0])
boxes.append(curr_ann[1]['yellow_block'][0])
boxes.append(curr_ann[1]['green_block'][0])
boxes.append(curr_ann[1]['blue_block'][0])
boxes.append(curr_ann[1]['red_block'][0])
input_boxes = torch.tensor(boxes, device=predictor.device)
transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])

points = []
points.append(curr_ann[1]['gripper_center'][0][:2])
points.append(curr_ann[1]['table_center'][0][:2])
points.append(curr_ann[1]['yellow_block_center'][0][:2])
points.append(curr_ann[1]['green_block_center'][0][:2])
points.append(curr_ann[1]['blue_block_center'][0][:2])
points.append(curr_ann[1]['red_block_center'][0][:2])
points_input = np.array([points])
for i in range(1, len(points)):
    points_input = np.append(points_input, [points], axis=0)
points_input = torch.tensor(points_input, device=predictor.device)
input_points = predictor.transform.apply_coords_torch(points_input, image.shape[:2])
labels = torch.diag(torch.ones(len(points), device=predictor.device))
first_frame_segs, scores, logits = predictor.predict_torch(
    point_coords=input_points,
    point_labels=labels,
    boxes = transformed_boxes,
    multimask_output = False,
)
print(first_frame_segs.shape)

# get the masks from the first frame
first_frame_masks = first_frame_segs.cpu().numpy()
first_frame_masks = first_frame_masks > 0.5
first_frame_masks = first_frame_masks.squeeze(1)
print(first_frame_masks.shape)

# combine the masks
combined_mask = combine_all_masks(first_frame_masks)

plt.figure(figsize=(big_plot_dim, big_plot_dim))
# plt.imshow(image)
show_mask(combined_mask, plt.gca())
plt.axis('off')
plt.show()

# get the largest contiguous section of each mask
for i in range(len(first_frame_masks)):
    first_frame_masks[i] = get_largest_contiguous(first_frame_masks[i])

combined_mask = combine_all_masks(first_frame_masks)

plt.figure(figsize=(big_plot_dim, big_plot_dim))
# plt.imshow(image)
show_mask(combined_mask, plt.gca())
plt.axis('off')
plt.show()

total_masks_over_time = np.array([first_frame_masks])
print(total_masks_over_time.shape)

## Segmentations over time of the rest of the images

In [ ]:
# figure out how many images there are in the directory
import os

image_files = os.listdir(image_dir)
image_files = [f for f in image_files if f.endswith('.jpg')]
image_files = sorted(image_files)
n_images = len(image_files)
print(f"Found {n_images} images in the directory")

In [ ]:
box_margin = 0.2

# total_masks_over_time = np.array([[relevant_first_mask_anns[i]['segmentation'] for i in range(len(relevant_first_mask_anns))]])
# # the shape of this is [frame_num, mask_num, height, width]
# print(total_masks_over_time.shape)
total_masks_over_time = np.array([first_frame_masks])
print(total_masks_over_time.shape)
for i in range(1, n_images):
    frame_num = i
    image = cv2.imread(image_dir + image_prefix + str(frame_num) + image_suffix)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    predictor.set_image(image)

    # get bounding boxes from previous masks
    boxes = [get_box(mask) for mask in total_masks_over_time[frame_num-1]]
    print("Frame Number: ", frame_num)
    # print(boxes)
    # inflate boxes
    for box in boxes:
        if box is not None:
            box[0] = max(0, box[0] - box_margin * (box[2] - box[0]))
            box[1] = max(0, box[1] - box_margin * (box[3] - box[1]))
            box[2] = min(image.shape[1], box[2] + box_margin * (box[2] - box[0]))
            box[3] = min(image.shape[0], box[3] + box_margin * (box[3] - box[1]))

    input_boxes = torch.tensor(boxes, device=predictor.device)

    transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])

    # commented out since just the boxes give us terrible performance and we use the boxes later anyways
    masks, scores, logits = predictor.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=transformed_boxes,
        multimask_output=True,
    )

    masks = masks.cpu().numpy()
    curr_frame_first_masks = np.array([masks[i][j] for i in range(masks.shape[0]) for j in range(masks.shape[1])])

    centroids = np.array([find_centroid(mask) for mask in total_masks_over_time[frame_num-1]])
    masks_points = torch.tensor(centroids, device=predictor.device)
    points_for_display = masks_points.cpu().numpy()
    # reshape masks_points to be of shape (n_masks, 1, 2)
    masks_points = masks_points.unsqueeze(1)
    masks_labels = torch.ones(len(masks_points), device=predictor.device)
    labels_for_display = masks_labels.cpu().numpy()
    # reshape masks_labels to be of shape (n_masks, 1)
    masks_labels = masks_labels.unsqueeze(1)
    input_points = predictor.transform.apply_coords_torch(masks_points, image.shape[:2])
    # print("Masks Points: ")
    # print(masks_points.shape)
    mask_point_segs, scores, logits = predictor.predict_torch(
        point_coords=input_points,
        point_labels=masks_labels,
        boxes=transformed_boxes,
        multimask_output=True,
    )
    mask_point_segs = mask_point_segs.cpu().numpy()
    curr_frame_point_seg_masks = np.array([mask_point_segs[i][j] for i in range(mask_point_segs.shape[0]) for j in range(mask_point_segs.shape[1])])
    # print("Mask Point Segs:")
    # print(mask_point_segs.shape)
    # print("Curr Frame Point Seg Masks:")
    # print(curr_frame_point_seg_masks.shape)

    points_input = np.array([centroids])
    for i in range(1, len(centroids)):
        points_input = np.append(points_input, [centroids], axis=0)
    labels = torch.diag(torch.ones(len(centroids), device=predictor.device))
    points_input = torch.tensor(points_input, device=predictor.device)
    input_points = predictor.transform.apply_coords_torch(points_input, image.shape[:2])
    third_seg_masks, scores, logits = predictor.predict_torch(
        point_coords=input_points,
        point_labels=labels,
        boxes=None,
        multimask_output=True,
    )
    third_seg_masks = third_seg_masks.cpu().numpy()
    curr_frame_third_seg_masks = np.array([third_seg_masks[i][j] for i in range(third_seg_masks.shape[0]) for j in range(third_seg_masks.shape[1])])

    # plt.figure(figsize=(big_plot_dim, big_plot_dim))
    # # plt.imshow(image)
    # k = 0
    # for mask in third_seg_masks:
    #     # print(mask.shape)
    #     # get first element of mask but keep shape of mask
    #     mask = mask[1]
    #     # print(mask.shape)
    #     show_mask(mask, plt.gca(), color=np.array([color_list[k]]))
    #     k = k + 1
    # for box in input_boxes:
    #     show_box(box.cpu().numpy(), plt.gca())
    # for point in input_points:
    #     show_points(points_for_display, labels_for_display, plt.gca())
    # plt.axis('off')
    # plt.show()

    use_contiguous = True
    use_first_segmentation = False
    # flatten all multimasks into one masks array
    # create curr_frame_all_masks as numpy array with dimensions n_masks, height, width
    if use_first_segmentation:
        curr_frame_all_masks = np.array([curr_frame_first_masks[i] for i in range(len(curr_frame_first_masks))])
        curr_frame_all_masks = np.append(curr_frame_all_masks, curr_frame_point_seg_masks, axis=0)
    else:
        curr_frame_all_masks = np.array([curr_frame_point_seg_masks[i] for i in range(len(curr_frame_point_seg_masks))])
    # merge curr_frame_all_masks with curr_frame_point_seg_masks
    curr_frame_all_masks = np.append(curr_frame_all_masks, curr_frame_third_seg_masks, axis=0)
    # get contiguous sections of each mask
    contiguous_all_masks = np.array([get_largest_contiguous(mask) for mask in curr_frame_all_masks])
    # print("Curr Frame All Masks:")
    # print(curr_frame_all_masks.shape)
    # compute matching pairs between masks and total_masks_over_time[frame_num]
    if not use_contiguous:
        matching_pairs = get_matching_masks(total_masks_over_time[frame_num-1], curr_frame_all_masks)
        matching_masks = [curr_frame_all_masks[pair[1]] for pair in matching_pairs]
    else:
        matching_pairs = get_matching_masks(total_masks_over_time[frame_num-1], contiguous_all_masks)
        matching_masks = [contiguous_all_masks[pair[1]] for pair in matching_pairs]
    print("Matching Pairs:")
    print(matching_pairs)
    total_masks_over_time = np.append(total_masks_over_time, np.array([matching_masks]), axis=0)
    print(total_masks_over_time.shape)

### Visualization

In [ ]:
# Visualize the total_masks_over_time as an animation, with each mask in each frame being a different color, side by side with the real image, for comparison
for i in range(0, n_images):
    image = cv2.imread(image_dir + image_prefix + str(i) + image_suffix)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(big_plot_dim*2, big_plot_dim*2))
    plt.subplot(1, 2, 1)
    plt.title(f"Frame {i} Real Image")
    plt.imshow(image)
    plt.subplot(1, 2, 2)
    plt.title(f"Frame {i} Segmentation")
    show_all_masks = True
    masks_to_show = [2]
    for j in range(len(total_masks_over_time[i])):
        if show_all_masks:
            show_mask(total_masks_over_time[i][j], plt.gca(), color=np.array(color_list[j]))
        elif j in masks_to_show:
            show_mask(total_masks_over_time[i][j], plt.gca(), color=np.array(color_list[j]))
    # output each frame in animation to file
    plt.savefig(f"frame_{i}.png")
    plt.show()
    time.sleep(0.1)
    clear_output(wait=True)

In [ ]:
# Visualize the total_masks_over_time, with each mask in each frame being a different color
# n = total_masks_over_time.shape[0]
# cols = 4
# rows = n // cols + 1
# plt.figure(figsize=(16, 16))
# for i in range(n):
#     plt.subplot(rows, cols, i + 1)
#     plt.title(f"Frame {i}")
#     for j in range(len(total_masks_over_time[i])):
#         show_mask(total_masks_over_time[i][j], plt.gca(), color=np.array(color_list[j]))
#     plt.axis('off')


In [ ]:
# Visualize all images in directory as animation
# for i in range(0, n_images):
#     image = cv2.imread(image_dir + image_prefix + str(i) + image_suffix)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     plt.figure(figsize=(big_plot_dim, big_plot_dim))
#     plt.title(f"Frame {i}")
#     plt.imshow(image)
#     plt.axis('off')
#     plt.show()
#     time.sleep(0.1)
#     clear_output(wait=True)

In [ ]:
# Visualize the total_masks_over time as an animation and in the same plot as the real image, for comparison
# for i in range(0, n_images):
#     image = cv2.imread(image_dir + image_prefix + str(i) + image_suffix)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     plt.figure(figsize=(big_plot_dim, big_plot_dim))
#     plt.title(f"Frame {i}")
#     plt.imshow(image)   # Comment this line to see only the masks
#     for j in range(len(total_masks_over_time[i])):
#         show_mask(total_masks_over_time[i][j], plt.gca(), color=np.array(color_list[j]))
#     plt.axis('off')
#     plt.show()
#     time.sleep(0.1)
#     clear_output(wait=True)

### Pickling necessary results for use with algorithmic tokenizer

In [ ]:
import pickle

out_path = image_dir + 'total_masks_over_time.pkl'
with open(out_path, 'wb') as f:
    pickle.dump(total_masks_over_time, f)